In [2]:
## Initialize
import spacy
from spacy.language import Language

import requests
import json
import numpy as np
import re

apiKey = "hTLGygty2Sj5IB368rcArA63Xu29hW2r"
archiveUrl = f'https://api.nytimes.com/svc/archive/v1/#1/#2.json?api-key={apiKey}'

def parseDoc(doc) -> list[str]:
  headline = doc["headline"]["print_headline"] or doc["headline"]["main"]
  leadPara = doc["lead_paragraph"]
  return str(headline + (" " if leadPara else "") + leadPara)

def custom_tokenizer(nlp):
    # Define a function to split tokens based on custom rules
    def split_tokens(doc):
        new_tokens = []
        for token in doc:
            if token.pos == ADJ or token.pos == NOUN:
                new_tokens.extend([t.text for t in nlp(token.text)])
            else:
                new_tokens.append(token.text)
        return Doc(nlp.vocab, words=new_tokens)
    
    return split_tokens

# Disabling components not needed (optional, but useful if run on a large dataset)
nlp = spacy.load("en_core_web_lg")
#nlp.tokenizer = custom_tokenizer(nlp)

nlp.add_pipe("merge_noun_chunks")
nlp.add_pipe("merge_entities")
#nlp.add_pipe("lemmatizer", config= {"mode": "lookup"})

# Load the larger model for similarity calculation
nlp_lg = nlp

# Example text
text = "German Chancellor Angela Merkel died in 1936 in New York. She got shot by a mysterious terrorist. terror is bad. terrorism is really not cool."

In [6]:
## Fetch

print(archiveUrl.replace("#1", "2010").replace("#2", "9"))
res = requests.get(archiveUrl.replace("#1", "2010").replace("#2", "9"))
obj = json.loads(res.text)
with open("september.json", "w", encoding="utf-8") as json_file:
    json_file.write(res.text)
#words = " ".join(map(lambda doc: parseDoc(doc), list(obj["response"]["docs"])))
# Test for clustering words in Caps
words = "HIDE KILLED PAKISTAN STATEMENT MALAYSIA CONGRESS PLOT ALABAMA SUITS STATEMENT MALAYSIA CONGRESS Killed".lower()

words


https://api.nytimes.com/svc/archive/v1/2010/9.json?api-key=hTLGygty2Sj5IB368rcArA63Xu29hW2r


'hide killed pakistan statement malaysia congress plot alabama suits statement malaysia congress killed'

In [95]:
## Search

def checkMatches(token, relevant_words):
    token_lg = nlp_lg(token.text)
    for comp in relevant_words.keys():
        comp_lg = nlp_lg(comp)
        similarity = token_lg.similarity(comp_lg)
        if(similarity >= 0.9):
            return (comp, similarity)
    return False

# Task 1: Extracting relevant words using the transformer-based model
wordCount = {}
matches: dict[str, dict] = {}
doc_trf = nlp(text)
for token in doc_trf:
    if token.text in matches.keys():
        matches[token.text]["amount"] += 1
    elif not token.is_stop and not token.is_punct:
        matched = checkMatches(token, wordCount)
        if(not matched):
            wordCount[token.text] = {}
            wordCount[token.text]["amount"] = 1
        else:
            if token.text in matches.keys():
                matches[token.text]["amount"] = matches[token.text]["amount"] + 1
            else:
                matches[token.text] = {
                    "match": matched[0],
                    # This might be removed later
                    "similarity": matched[1],
                    "amount": 1
                }




AttributeError: 'str' object has no attribute 'pos'

In [ ]:
for m in matches.items():
    if "duplicates" not in wordCount[m[1]["match"]]:
        wordCount[m[1]["match"]]["duplicates"] = {}
    wordCount[m[1]["match"]]["duplicates"][m[0]] = {
        "amount": m[1]["amount"],
        "similarity": m[1]["similarity"]
    }
    wordCount[m[1]["match"]]["amount"] = wordCount[m[1]["match"]]["amount"] + m[1]["amount"]

wordCount

{'German Chancellor Angela Merkel': {'amount': 1},
 'died': {'amount': 1},
 '1936': {'amount': 1},
 'New York': {'amount': 1},
 'got': {'amount': 1},
 'shot': {'amount': 1},
 'a mysterious terrorist': {'amount': 1},
 'terror': {'amount': 1},
 'bad': {'amount': 1},
 'terrorism': {'amount': 1},
 'cool': {'amount': 1}}

In [ ]:
print(wordCount)
print("\n--------------------\n")
print(matches)

{'German Chancellor Angela Merkel': {'amount': 1}, 'died': {'amount': 1}, '1936': {'amount': 1}, 'New York': {'amount': 1}, 'got': {'amount': 1}, 'shot': {'amount': 1}, 'a mysterious terrorist': {'amount': 1}, 'terror': {'amount': 1}, 'bad': {'amount': 1}, 'terrorism': {'amount': 1}, 'cool': {'amount': 1}}

--------------------

{}


In [ ]:
import spacy
taggers = [{
    "name": "Taggers",
    "values": [
        "$", "''", ",", "-LRB-", "-RRB-", ".", ":", "ADD", "AFX", "CC", "CD", "DT", "EX", "FW", "HYPH", "IN", "JJ", "JJR", "JJS", "LS", "MD", "NFP", "NN", "NNP", "NNPS", "NNS", "PDT", "POS", "PRP", "PRP$", "RB", "RBR", "RBS", "RP", "SYM", "TO", "UH", "VB", "VBD", "VBG", "VBN", "VBP", "VBZ", "WDT", "WP", "WP$", "WRB", "XX", "_SP", "``"
    ]
}]

for tagger in taggers:
    print(tagger["name"])
    print(tagger["values"])

for tagger in taggers:

    for tag in tagger["values"]:
        print(f"{tag["name"]}: {spacy.explain(tag)}")

Taggers
['$', "''", ',', '-LRB-', '-RRB-', '.', ':', 'ADD', 'AFX', 'CC', 'CD', 'DT', 'EX', 'FW', 'HYPH', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NFP', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', 'XX', '_SP', '``']


TypeError: string indices must be integers, not 'str'

In [103]:
x = nlp_lg(text)
for i in x:
    for j in x:
        if i.similarity(j) >0.4 and i.similarity(j)<1:
            print(i,j, i.similarity(j))

for i in x:
    print(i.lemma_)


a mysterious terrorist terror 0.4993249475955963
a mysterious terrorist terrorism 0.5030153393745422
a mysterious terrorist really 0.4049932062625885
terror a mysterious terrorist 0.4993249475955963
terror terrorism 0.8408539891242981
terrorism a mysterious terrorist 0.5030153393745422
terrorism terror 0.8408539891242981
really a mysterious terrorist 0.4049932062625885
really not 0.5845474600791931
really cool 0.4084397852420807
not really 0.5845474600791931
cool really 0.4084397852420807
German Chancellor Angela Merkel
die
in
1936
in
New York
.
she
get
shoot
by
a mysterious terrorist
.
terror
be
bad
.
terrorism
be
really
not
cool
.


In [ ]:
my_text = "i ate food graciously. he has eaten. she eats. they eat"
my_text= "there was terror. terrorism is bad. there were terrorists"
x = nlp(my_text)
for i in x:
    print(i.lemma_, i.pos_)

there PRON
be VERB
terror NOUN
. PUNCT
terrorism NOUN
be AUX
bad ADJ
. PUNCT
there PRON
be VERB
terrorist NOUN


In [102]:
nlp = spacy.load("en_core_web_lg")
my_text = "German Chancellor Angela Merkel died in 1936 in New York. She got shot by a mysterious terrorist. terror is bad. terrorism is really not cool."
doc = nlp(my_text)

for token in doc:
    print(token.text, token.pos_)

German PROPN
Chancellor PROPN
Angela PROPN
Merkel PROPN
died VERB
in ADP
1936 NUM
in ADP
New PROPN
York PROPN
. PUNCT
She PRON
got AUX
shot VERB
by ADP
a DET
mysterious ADJ
terrorist NOUN
. PUNCT
terror NOUN
is AUX
bad ADJ
. PUNCT
terrorism NOUN
is AUX
really ADV
not PART
cool ADJ
. PUNCT
